# Imports

In [1]:
from tqdm import tqdm
import joblib
import re
import spacy
import nltk
import pandas as pd

import sys
import json
#import stanza
#stanza.download('pt')
import numpy as np
# install and import amr utils
sys.path.append('../')
sys.path.append("../src/")
from features.nlp_preprocess import *
#!pip install ../amr-utils
from amr_utils.amr_readers import AMR_Reader

In [2]:
# folder = '../POS-tagger-portuguese-nltk/trained_POS_taggers/'
# tagger_nltk = joblib.load(folder+'POS_tagger_brill.pkl')

# Read data

In [3]:
# ao salvar em json ha algumas desformatacoes, vamos consertar 
def convert_keys_to_int(data):
    if 'annotated_sentence' in data.keys():
        data['annotated_sentence'] = {int(k): v for k, v in data['annotated_sentence'].items()}
    else: print("Errro")
    return data

# with open('../data/processed/annotated_text.json', 'r') as f:
#     list_dict_an = json.loads(f.read())

with open('../data/processed/annotated_text.json', 'r') as f:
    list_dict_an = json.loads(f.read())
    
with open('../data/processed/annotated_text_propbank.json', 'r') as f:
    list_dict_an_propbank = json.loads(f.read())
    
    
list_dict_an = [convert_keys_to_int(dict) for dict in list_dict_an]
list_dict_an_propbank = [convert_keys_to_int(dict) for dict in list_dict_an_propbank]

In [4]:
print(len(list_dict_an))
print(len(list_dict_an_propbank))

2959
3348


In [5]:
list_dict_an[0]

{'id': '1',
 'nodes': {'1': 'ter-01',
  '1.1': 'person',
  '1.1.1': 'name',
  '1.1.2': 'explicar-01',
  '1.1.2.2': 'coisa',
  '1.1.2.2.1': 'esse',
  '1.1.2.2.2': 'resultar-01',
  '1.1.1.1': '"Meyer"',
  '1.1.2.3': '2'},
 'edges': {'edge 1': {'nodes_ids': ['1.1.1', '1.1.1.1'],
   'nodes': ['name', '"Meyer"'],
   'value': ':op1'},
  'edge 2': {'nodes_ids': ['1.1.2', '1.1.2.3'],
   'nodes': ['explicar-01', '2'],
   'value': ':quant'},
  'edge 3': {'nodes_ids': ['1', '1.1'],
   'nodes': ['ter-01', 'person'],
   'value': ':ARG0'},
  'edge 4': {'nodes_ids': ['1.1', '1.1.1'],
   'nodes': ['person', 'name'],
   'value': ':name'},
  'edge 5': {'nodes_ids': ['1.1', '1.1.2'],
   'nodes': ['person', 'explicar-01'],
   'value': ':ARG1'},
  'edge 6': {'nodes_ids': ['1.1.2', '1.1'],
   'nodes': ['explicar-01', 'person'],
   'value': ':ARG0'},
  'edge 7': {'nodes_ids': ['1.1.2', '1.1.2.2'],
   'nodes': ['explicar-01', 'coisa'],
   'value': ':ARG1'},
  'edge 8': {'nodes_ids': ['1.1.2.2', '1.1.2.2.1'],


In [6]:
list_dict_an_original = list_dict_an.copy()

In [7]:
list_dict_an =  list_dict_an + list_dict_an_propbank
#list_dict_an =  list_dict_an_propbank

In [8]:
len(list_dict_an)

6307

# Create dataframe

In [9]:
# pos que não podem estar contidas no amr (descritas no artigo)
# punct tambem removi pq tbm nao pode e esta no artigo de certa forma
pos_filter = ["SCONJ", "DET", "ADJ", "PUNCT"]

In [10]:
def find_nodes_ids(
    annotated_sentence,
    edge_amr_info
):
    
    parent, child = edge_amr_info['nodes']
    
    
    parent = remove_num_text(parent)
    child = remove_num_text(child) 
    
    

    if parent != None:
        
        # acha a posicao de todos os tokens no texto que sao iguais a parent
        parent_w_app = [id for id, an in annotated_sentence.items() if (an["lemma"].casefold() == parent.casefold() or an["text"].casefold() == parent.casefold()) and an["pos"] not in pos_filter]
        
        # todo nome pessoal vira person na anotação, então precisa achar qual nome tem de pessoa tem no texto
        if len(parent_w_app) == 0 and parent == "person":
            
            parent_w_app = [id for id, an in annotated_sentence.items() if an["ner"] == "PER"]
            
            if len(parent_w_app) >1:
                
                # nomes compostos vao ocupar dois tokens. Para verificar se é um nome composto parta olha o inicio e fim do ner
                inicio_fim = annotated_sentence[parent_w_app[0]]['ner_start_end'] # basta pegar o inicio e fim do primeiro token
                
                # verifica se o inicio e fim bate com os ids encontrados
                contains = True
                for i in parent_w_app:
                    if i not in range(inicio_fim[0], inicio_fim[1]):
                        contains = False
                

                if contains:
                    
                    parent_w_app = [tuple(parent_w_app)]
        
    else:

        parent_w_app = None

    if child != None:
        
        # acha a posicao de todos os tokens no texto que sao iguais a child
        child_w_app = [id for id, an in annotated_sentence.items() if (an["lemma"].casefold() == child.casefold() or  an["text"].casefold() == child.casefold()) and an["pos"] not in pos_filter]

        # todo nome pessoal vira person na anotação, então precisa achar qual nome tem de pessoa tem no texto
        if len(child_w_app) == 0 and child == "person":
            
            child_w_app = [id for id, an in annotated_sentence.items() if an["ner"] == "PER"]
            
            if len(child_w_app) >1:
                
                # nomes compostos vao ocupar dois tokens. Para verificar se é um nome composto parta olha o inicio e fim do ner
                inicio_fim = annotated_sentence[child_w_app[0]]['ner_start_end'] # basta pegar o inicio e fim do primeiro token
                
                # verifica se o inicio e fim bate com os ids encontrados
                contains = True
                for i in child_w_app:
                    if i not in range(inicio_fim[0], inicio_fim[1]):
                        contains = False
                

                if contains:
                    
                    child_w_app = [tuple(child_w_app)]
                
        
    else: 
        
        child_w_app = None
        
    len_parent_w_app = len(parent_w_app) if parent_w_app is not None else None
    len_child_w_app = len(child_w_app) if child_w_app is not None else None
    
    if len_parent_w_app == 1 and len_child_w_app == 1:

        
        # uso de set para não importar a ordem da tupla
        pair = (parent_w_app[0], child_w_app[0])
        parent_id, child_id = pair
        
        if type(parent_id) != tuple:
            parent_to_child = False
            if annotated_sentence[parent_id]["head_index"] == child_id:
                parent_to_child = True
                
        else:
            parent_to_child = False
            for id in parent_id:
                if annotated_sentence[id]["head_index"] == child_id:
                    parent_to_child = True 
            
        
        if type(child_id) != tuple:
            child_to_parent = False
            if annotated_sentence[child_id]["head_index"] == parent_id:
                child_to_parent = True
        else:
            child_to_parent = False
            for id in child_id:
                if annotated_sentence[id]["head_index"] == parent_id:
                    child_to_parent = True                 

            
        return {"response": {"pair":pair, "dep_parent_to_child": parent_to_child, "dep_child_to_parent": child_to_parent, "type": f"parent_{len_parent_w_app}_child_{len_child_w_app}"}}
    
    # se achou 1 e outro não. Vale também para quando um deles não existe (None)
    if (len_parent_w_app == 1 and len_child_w_app == 0) or (len_parent_w_app == 0 and len_child_w_app == 1) or (len_parent_w_app == 1 and len_child_w_app == None) or (len_parent_w_app == None and len_child_w_app == 1): 
        
        if len(parent_w_app) == 0:
            
            parent_id = None
        else: 
            
            parent_id = parent_w_app[0]
        if len(child_w_app) == 0:
            child_id = None
        else:
            child_id = child_w_app[0]
        
        pair = (parent_id, child_id)
        parent_to_child = False
        child_to_parent = False
        return {"response": {"pair":pair, "dep_parent_to_child": parent_to_child, "dep_child_to_parent": child_to_parent, "type": f"parent_{len_parent_w_app}_child_{len_child_w_app}"}}
        #return "miss_one"
        
        
    
    return {"response": {"pair":(None, None), "dep_parent_to_child": None, "dep_child_to_parent": None, "type": f"parent_{len_parent_w_app}_child_{len_child_w_app}"}}
def get_dependency_direction(
    annotated_sentence,
    parent_id, 
    child_id
):
    
    parent_to_child = False     
    if parent_id is not None:
        if annotated_sentence[parent_id]["head_index"] == child_id:
            parent_to_child = True
     
    child_to_parent = False       
    if child_id is not None:
        if annotated_sentence[child_id]["head_index"] == parent_id:
            child_to_parent = True
        
    return parent_to_child, child_to_parent

import os


list_rows = []
list_errors_match = [] #(debug)
list_n_match = [] # (debug)
for dict_an in tqdm(list_dict_an):
    
    try:
    
        edges = dict_an['edges']

        # create dep pairs
        dep_pairs = []
        for tk_id, an in dict_an['annotated_sentence'].items():
            p = an
            p_head_id = an['head_index']
            p_head = dict_an['annotated_sentence'].get(p_head_id)
            # nao salva os pares que possuem os pos que nao podem ser amr
            if p['pos'] in pos_filter or p['pos'] in p_head: continue
            
            dep_pairs.append((tk_id,p_head_id))

        for edge_id, edge_info in edges.items():
            
            parent, child = edge_info['nodes']
            
            value = edge_info['value']
            
            if "arg" not in value.casefold(): continue
            
            if dict_an['dict_aligments'] is None:
                
                
                # acha os ids dos nós
                response = find_nodes_ids(
                annotated_sentence = dict_an['annotated_sentence'],
                edge_amr_info = edge_info)
                
                pair = response['response']['pair']
                
                
                parent_id, child_id = pair
                    
                
                dep_parent_to_child = response['response']['dep_parent_to_child']
                dep_child_to_parent = response['response']['dep_child_to_parent']
                
                
                type_response = response['response']['type']
                
                
            else:
                
                parent_id = dict_an['dict_aligments'].get(edge_info['nodes_ids'][0], None)
                child_id = dict_an['dict_aligments'].get(edge_info['nodes_ids'][1], None) 

                
                pair = (parent_id, child_id)
                
                dep_parent_to_child, dep_child_to_parent = get_dependency_direction(annotated_sentence=dict_an['annotated_sentence'], parent_id=parent_id, child_id=child_id)
                
                type_response = "match_aligments"

            ann_sent = dict_an['annotated_sentence']
            
            if not dep_parent_to_child and  not dep_child_to_parent:
                #print("dep", dep_parent_to_child, dep_child_to_parent)
                dep = "nao_tem_dep"
                
            else: 
                
                if dep_parent_to_child and not dep_child_to_parent:
                    
                    dep = ann_sent[child_id]['dep']
                    
                elif dep_child_to_parent and not dep_parent_to_child:
                    
                    if type(child_id) != tuple:
                    
                        dep = ann_sent[child_id]['dep']
                    else:
                        dep_list = []
                        for id in child_id:
                            dep_list.append(ann_sent[id]['dep'])
                            
                        # remove a dep que indica que o token é continuação de um nome (ex: anotonio eduardo, eduardo tem dep flat:name)
                        dep_list = [dep for dep in dep_list if dep != "flat:name"]
                        
                        
                        if len(dep_list) == 1:
                            dep = dep_list[0]
                        else:
                            raise Exception("erro")    
                        
                        
                        
                else:
                    # a palavra tem dependencia com ela mesma
                    if parent_id == child_id:
                        
                        # pega a dependencia (nao importa se pega do pai ou do filho)
                        dep = ann_sent[child_id]['dep']
                        
                    else:
                    
                        raise Exception("Há dependencia de duas mãos") 
                    
                    
                                
            #### lembrar de colocar o parent como o token do parent, nao oq veio do amr
            new_row = {
                "corpus_name": dict_an['corpus_name'],
                "id": dict_an['id'],
                "parent": remove_num_text(parent) if parent is not None else None,
                "child":remove_num_text(child) if child is not None else None,
                "label": edge_info["value"],
                "dep": dep
            }
            
            # anotacoes inuteis
            useless_cols = ["head_index", "dep", "id"]
            
            if parent_id is not None:
                
                if type(parent_id) != tuple:
                    ann_parent = ann_sent[parent_id]
                    dict_parent = {f'parent_{key}':value for key, value in ann_parent.items() if key not in useless_cols}
                    
                else:
                    
                    for key, value in ann_parent.items():
                        
                        if key not in useless_cols:
                            
                            values = [ann_sent[id][key] for id in parent_id]
                            
                            # checa se todos os elementos são iguais
                            if all(i == values[0] for i in values):
                                # todos os elementos sao iguais
                                final_value = values[0]
                            else:
                                # sao diferentes
                                final_value = " ".join(values)
                                
                            
                            dict_parent.update({f'parent_{key}': final_value})
                
            else: 
                dict_parent = {f'parent_{key}':"token_nao_encontrado_no_texto" for key in ['text', 'lemma', 'pos', 'tag', 'shape', 'is_alpha', 'is_stop', 'morph', 'ner']}
            
            
            # cria dict com as features
            if child_id is not None:
                
                if type(child_id) != tuple:
                    ann_child = ann_sent[child_id]
                    dict_child = {f'child_{key}':value for key, value in ann_child.items() if key not in useless_cols}
                    
                else:
                    
                    for key, value in ann_child.items():
                        
                        if key not in useless_cols:
                            
                            values = [ann_sent[id][key] for id in child_id]
                            
                            # checa se todos os elementos são iguais
                            if all(i == values[0] for i in values):
                                # todos os elementos sao iguais
                                final_value = values[0]
                            else:
                                # sao diferentes
                                final_value = " ".join([str(value) for value in values])
                                
                            
                            dict_child.update({f'child_{key}': final_value})
                
            else:
                dict_child = {f'child_{key}':"token_nao_encontrado_no_texto" for key in ['text', 'lemma', 'pos', 'tag', 'shape', 'is_alpha', 'is_stop', 'morph', 'ner']}


            new_row.update(dict_parent)
            new_row.update(dict_child)        
            list_rows.append(new_row)
            list_errors_match.append(type_response)
    except Exception as e:
        print(e)
        
                  
        
df = pd.DataFrame(list_rows)
df

100%|██████████| 6307/6307 [00:00<00:00, 9285.11it/s] 


,corpus_name,id,parent,child,label,dep,parent_text,parent_lemma,parent_pos,parent_tag,...,child_lemma,child_pos,child_tag,child_shape,child_is_alpha,child_is_stop,child_morph,child_ner,child_ner_start_end,parent_ner_start_end
0,sci,1,ter,person,:ARG0,nsubj,tem,ter,VERB,VERB,...,Meyer,PROPN,PROPN,Xxxxx,True,False,Gender=Masc|Number=Sing,PER,"[0, 1]",NaN
1,sci,1,person,explicar,:ARG1,nao_tem_dep,Meyer,Meyer,PROPN,PROPN,...,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,NaN,"[0, 1]"
2,sci,1,explicar,person,:ARG0,nao_tem_dep,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,...,Meyer,PROPN,PROPN,Xxxxx,True,False,Gender=Masc|Number=Sing,PER,"[0, 1]",NaN
3,sci,1,explicar,coisa,:ARG1,nao_tem_dep,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,...,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,NaN,NaN
4,sci,1,coisa,resultar,:ARG2-of,nao_tem_dep,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,...,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,token_nao_encontrado_no_texto,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17217,PropBankBr_v1.1_Dep,propbank_3347,ser,Abram_Szajman,ARG1,nao_tem_dep,é,ser,AUX,AUX,...,Abram_Szajman,PROPN,PROPN,Xxxxx_Xxxxx,False,False,Gender=Masc|Number=Sing,LOC,"[0, 1]",NaN
17218,PropBankBr_v1.1_Dep,propbank_3347,ser,presidente,ARG2,ROOT,é,ser,AUX,AUX,...,presidente,NOUN,NOUN,xxxx,True,False,Gender=Masc|Number=Sing,None,NaN,NaN
17219,PropBankBr_v1.1_Dep,propbank_3348,surpreender,que,ARG0,nsubj,surpreendeu,surpreender,VERB,VERB,...,que,PRON,PRON,xxx,True,True,Gender=Masc|Number=Sing|PronType=Rel,None,NaN,NaN
17220,PropBankBr_v1.1_Dep,propbank_3348,surpreender,galera,ARG1,obj,surpreendeu,surpreender,VERB,VERB,...,galera,NOUN,NOUN,xxxx,True,False,Gender=Fem|Number=Sing,None,NaN,NaN


In [13]:
unique, counts = np.unique(list_errors_match, return_counts=True) 
list_count = np.asarray((unique, counts)).T
print(list_count)

[['match_aligments' '11250']
 ['parent_0_child_0' '1052']
 ['parent_0_child_1' '1309']
 ['parent_0_child_2' '77']
 ['parent_0_child_3' '8']
 ['parent_0_child_4' '2']
 ['parent_0_child_None' '1']
 ['parent_1_child_0' '1085']
 ['parent_1_child_1' '2212']
 ['parent_1_child_2' '113']
 ['parent_1_child_3' '15']
 ['parent_1_child_4' '5']
 ['parent_2_child_0' '23']
 ['parent_2_child_1' '46']
 ['parent_2_child_2' '10']
 ['parent_3_child_0' '3']
 ['parent_3_child_1' '4']
 ['parent_3_child_2' '5']
 ['parent_3_child_3' '1']
 ['parent_4_child_1' '1']]


In [18]:
match_alignments = []
match_1_1 = []
others_match = []
for case in list_count:
    
    if case[0] == 'match_aligments':
        match_alignments.append(int(case[1]))
    elif case[0] == 'parent_1_child_1':
        match_1_1.append(int(case[1]))
    else:
        others_match.append(int(case[1]))
        
print("match_alignments", sum(match_alignments))
print("match_1_1", sum(match_1_1))
print("others_match", sum(others_match))

match_alignments 11250
match_1_1 2212
others_match 3760


In [22]:
df.groupby('corpus_name').size()

corpus_name
PropBankBr_v1.1_Dep    10176
lp                      3763
news                    1471
opisums                 1397
sci                      415
dtype: int64

In [23]:
11250 - 10176

1074

In [ ]:
# dropar todas as linhas que tenham token_nao_encontrado_no_texto
df_filtered = df.drop(df.index[df.applymap(lambda x: x == 'token_nao_encontrado_no_texto').any(axis=1)])

/tmp/ipykernel_427611/544712622.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_filtered = df.drop(df.index[df.applymap(lambda x: x == 'token_nao_encontrado_no_texto').any(axis=1)])


In [ ]:
df_filtered.to_csv("../data/processed/processed_dataset.csv", index=False)

In [ ]:
df.to_csv("../data/processed/processed_dataset_unfiltered.csv", index=False)